Real-Life Example: "Email Assistant for Job Applications"
Scenario:

- You have a virtual assistant that:

1. Cleans up raw user input.
2. Generates a professional email using Gemini.
3. Optionally translates it if the user requests another language.

Workflow (Graph) Design
Nodes (functions):
- clean_input – Clean spelling, punctuation, tone.
- generate_email – Let Gemini generate a professional email.
- translate_output – (Optional) Translate the email to a user-specified language.

Edges (transitions):

- From clean_input → generate_email
- From generate_email → translate_output (if user requested)
- Otherwise, → END

In [12]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph, END
from pydantic import BaseModel
from typing import Optional

In [13]:
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

State class

In [14]:
class EmailState(BaseModel):
    raw_input: str
    cleaned_input: Optional[str] = None
    email: Optional[str] = None
    translated_email: Optional[str] = None
    target_language: Optional[str] = None

Node Or Agents

In [15]:
# Clean Input Node
def clean_input_node(state:EmailState):
    # Simple preprocessing (placeholder for real NLP)
    cleaned = state.raw_input.strip().capitalize()
    updated_state_data = state.model_dump() # instead of dict() we use model_dump
    updated_state_data['cleaned_input'] = cleaned
    return EmailState(**updated_state_data)

# Generate Email Node
def generate_email_node(state:EmailState):
    llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash",api_key=GOOGLE_API_KEY,temperature=0.7)
    prompt = f"Write a professional job application email based on this: '{state.cleaned_input}'"
    response = llm.invoke(prompt)
    updated_state_data = state.model_dump()
    updated_state_data['email'] = response.content
    return EmailState(**updated_state_data)

# Translate Output
def translate_node(state:EmailState):
    if not state.target_language:
        return state # No translation needed
    
    llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash",api_key=GOOGLE_API_KEY,temperature=0.7)
    prompt = f"Translate this email to {state.target_language}:\n{state.email}."
    response = llm.invoke(prompt)
    updated_state_data = state.model_dump()
    updated_state_data['translated_email'] = response.content
    return EmailState(**updated_state_data)

# Condition Function
def should_translate(state:EmailState):
    return "translate_output" if state.target_language else END

Build the Graph

In [16]:
graph = StateGraph(state_schema=EmailState)

graph.add_node("clean_input", clean_input_node)
graph.add_node("generate_email", generate_email_node)
graph.add_node("translate_output",translate_node)

# Transitions
graph.set_entry_point("clean_input")
graph.add_edge("clean_input", "generate_email")
graph.add_conditional_edges("generate_email",should_translate,{
    "translate_output":"translate_output",
    END:END
})
graph.add_edge("translate_output",END)

app = graph.compile()


Test the Graph

In [17]:
state = EmailState(raw_input="i want to apply for frontend dev job at google",target_language=None)
result = app.invoke(state)

print("📧 Final Email:\n",result['email'])

📧 Final Email:
 Subject: Application for Frontend Developer Position - [Your Name]

Dear Google Hiring Team,

I am writing to express my strong interest in Frontend Developer positions at Google, as advertised on [mention where you saw the advertisement - e.g., Google Careers website, LinkedIn, etc.]. I have been consistently impressed by Google's commitment to innovation and user-centric design, and I am confident that my skills and experience align well with your team's objectives.

For [Number] years, I have been working as a Frontend Developer, building and maintaining engaging and performant web applications. My expertise lies in [List 2-3 key skills and technologies - e.g., React, Angular, Vue.js, JavaScript (ES6+), HTML5, CSS3]. I have a proven track record of success in [Mention 1-2 key achievements or projects - e.g., developing responsive user interfaces, optimizing website performance, contributing to open-source projects].

In my previous role at [Previous Company Name], I 

Translate to french

In [19]:
state_fr = EmailState(raw_input="i want to apply for frontend dev job at google", target_language="French")
result_fr = app.invoke(state_fr)

print("📧 Final Email (French):\n",result_fr['translated_email'])

📧 Final Email (French):
 Here's the French translation of the email, incorporating the improvements and considerations you outlined:

**Objet : Candidature Développeur Frontend - [Your Name]**

Cher(ère) équipe de recrutement Google,

Je vous écris pour vous faire part de mon vif intérêt pour les postes de Développeur Frontend chez Google, tels que ceux annoncés sur [Platform where you saw the advertisement - e.g., Google Careers website, LinkedIn, etc.]. Je suis constamment impressionné(e) par l'engagement de Google en matière d'innovation et par son leadership dans le développement d'applications web de pointe, centrées sur l'utilisateur. Je suis convaincu(e) que mes compétences et mon expérience correspondent parfaitement aux besoins de votre équipe.

Depuis [Number] ans, je travaille en tant que Développeur Frontend, spécialisé(e) dans [Mention specific technologies and areas of expertise, e.g., React, Angular, Vue.js, JavaScript, HTML, CSS, responsive design, accessibility, perfor

✅ Summary of What You Just Learned:

| Component           | What It Represents            |
| ------------------- | ----------------------------- |
| `EmailState`        | Global shared memory          |
| `clean_input`       | Preprocess user message       |
| `generate_email`    | Core Gemini-powered response  |
| `translate_output`  | Optional post-processing node |
| `conditional_edges` | LangGraph's branching logic   |
